<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
# Import packages

import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from datetime import datetime

In [3]:
# Set base parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes"
LOAD_FILE = os.path.join(LOAD_PATH, "guideline_raw.json")

SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "guideline_structured.json")

# Error path and file name are defined below in orchestrate function to allow time-stamp

In [ ]:
# Load LLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [5]:
# Check GPU connection

import torch
print(torch.cuda.is_available())
print(model.device)

True
cuda:0


In [ ]:
# Test

## Should also carry out test prompt of transforming recommendations?

text = "Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable?"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=500)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable? No, not necessarily. Lithium is a mood stabilizer commonly used in the treatment of bipolar disorder, and it can help manage symptoms such as mania, hypomania, depression, and irritability. Mild irritability alone does not typically warrant a referral to secondary care.

However, it's important to monitor the individual's overall mental health status and any changes in their condition. If the irritability persists or worsens, or if there are other concerning symptoms (such as significant mood swings, suicidal thoughts, psychotic features, or side effects from the medication), then a referral to secondary care might be appropriate for a more comprehensive evaluation and management plan.

Regular follow-ups with a primary care provider or a psychiatrist are crucial to ensure that the treatment plan is effective and safe. They can adjust the medication dosage, add o

In [6]:
# Load JSON of raw recommendations

def load_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {file_path}')

raw_recommendations = load_json(LOAD_FILE)

print(type(raw_recommendations))
print(len(raw_recommendations))
print(raw_recommendations[0])

<class 'list'>
136
{'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': None, 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.'}


In [ ]:
# Old version: trying to separate out all conditions

def construct_prompt(entity):

    # Need to do manual read-through of entire guideline to check value fidelity
    # Risk: maybe risk_to_others hasn't been populated at all, is that correct?
    # Work on physical health fields to increase specificity?
    # current_manic_phase is only populating occasionally with "mania" and not other values: incorrect?

    """
    Given one recommendation entry {}, creates the prompt to extract one normalised JSON item
    """

    heading_1 = entity.get('heading_1')
    sub_heading_1 = entity.get('sub_heading_1')
    sub_heading_2 = entity.get('sub_heading_2')

    original_recommendation_number = entity.get('original_recommendation_number')
    original_recommendation_text = entity.get('original_recommendation_text')

    heading_context = " > ".join(h.strip() for h in [heading_1, sub_heading_1, sub_heading_2] if isinstance(h, str) and h.strip())

    return f"""
    You are extracting structured information from a NICE guideline recommendation.

    RULES:
    - output must be valid JSON ONLY (no markdown; no commentary)
    - do not invent clinical information, thresholds or populations; use only what is present in the recommendation text
    - For ALL clinical descriptor fields (e.g. phase, severity, medication): populate a value ONLY if it is explicitly stated in the recommendation text. Do NOT infer information that is not directly stated. If not explicit, use null.
    - 'conditionality' must be a string describing the constraint or constraints on the implementation of a recommended clinical action, indicated by clauses such as 'if...', 'where...', 'when...' or 'who are...' (e.g., 'if a person has rapid or excessive weight gain, abnormal lipid levels or problems with blood glucose management'). If the text says ‘If [condition], do X’, conditionality must include the entire [condition] clause (including outcomes like response/no response), even if it contains time windows. Do not treat time windows (“within 4 weeks…”, “after 4–6 weeks…”) as conditionality unless they are part of an if/when trigger. If none is mentioned, record null. If multiple conditions, return each as a string in a list. Do not include recommendation 'action' content, which occurs at the boundary before the action verb.
    - 'conditionality_type' must be one of: ['temporal_trigger', 'prohibition', 'treatment_stage', 'prescribed_medication', 'side_effects', 'treatment_response', 'test_results', 'patient_preference', null]. If more than one applies, record all as a list of strings. 'prescribed_medication' applies only if the person is currently taking a named medication. 'treatment_stage' indicates start/stop/change/adjust/monitor language.
    - 'conditionality_operator' must show how multiple conditionality items are logically combined. It must be 'and' if all conditions apply, or 'or' if one condition is sufficient. If only one condition is present or 'conditionality' is null, record null. If the structure is mixed or unclear, record 'undetermined'.
    - 'action' must be a verb phrase indicating the clinical decision or intervention (e.g., 'continue treatment and care in an early intervention in psychosis service, a specialist bipolar disorder service or a specialist integrated community-based team', monitor closely for signs of depression...) that is being done to/for a person. It should capture the clinical instruction and not merely the verb (e.g., 'ensure that people have access to calming environments and reduced stimulation', not just 'ensure'). If more than one action is mentioned, retain all as a list of strings. When an action includes a quantitative target (dose, plasma level, frequency, duration), include that target in the action string. Do NOT include information giving (indicated by 'explain', 'discuss' or similar verbs): This should be stored in 'information_giving'. Do not include if/when information relevant to conditionality.
    - 'information_giving' must be a verb phrase indicating sharing information, explaining or discussing with the patient. If not mentioned, set to null.
    - 'temporal_constraints' must capture additional requirements attached to an action, such as timing, frequency, duration or review intervals that affects an action (e.g., 'within 4 weeks of symptom resolution', 'for a trial period of 6 months'). If no temporal_constraints are mentioned, record null.
    - Extract 'prohibitions_and_cautions' from verb phrases including 'do not', 'must not' or 'should not', and clauses beginning 'but...'. If none are mentioned, record null.
    - 'population' is the patient or staff group that the recommendation applies to (e.g., 'people with bipolar…', 'pregnant women…', 'psychological therapists', etc.). If none is mentioned, record null.
    - 'age_group' must be one of: ['child', 'young_person', 'older_adult', 'adult', null]
    - Extract 'urgency' as True if the text includes 'urgent', 'urgently', 'immediate' or 'immediately', otherwise False
    - 'manic_episode_history' must be one of: ['none', 'one', 'multiple', null]
    - 'current_manic_phase' must be one of: ['mania', 'hypomania', 'depression', 'mixed', 'rapid_cycling', 'euthymic', null]
    - 'symptom_severity' must be one of: ['mild', 'moderate', 'severe', null]
    - 'current_psychosis' must be one of: ['present', 'absent', null]
    - 'diagnoses' must be one or more mental health diagnoses. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
    - 'prescribed_medication' should be populated by medication names if the recommendation states explicitly that the patient is taking them currently, or that the recommendation only applies to patients who are taking a medication. Otherwise set it to null and record any medications in 'suggested_medications'. If there are multiple medication names, retain each as a list of strings.
    - 'medication_adherence' must be one of: ['good', 'poor', null]
    - 'side_effects' is a string describing side effects mentioned in the recommendation, or is set to null.
    - 'suggested_treatment' should be populated by suggested treatment actions (medication names, psychological therapies, exercise programmes, electroconvulsive therapy), or set to null. If there are treatments suggested, retain each as a list of strings. If the medications are to be taken together (as suggested by 'combined'), record as a single string.
    - 'treatment_stage' must be one of: ['start', 'stop', 'continue', 'change', 'adjust', 'monitor', null]
    - 'psychological_therapy_type' must be one of: ['psychodynamic', 'psychoeducation', 'cognitive_behavioural', 'other', null]
    - 'psychological_therapy_delivery' must be one of: ['individual', 'group', null]
    - 'attitude_towards_psychological_therapy' must be one of: ['positive', 'ambivalent', 'negative', null]
    - 'physical_health_longterm' must be the name of a physical disease diagnosis that usually affects a person for more than six months. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
    - 'physical_health_recent' must be the name of a transient disease (less than six months) or physical health event from the last six months. Treat phrases like ‘impaired renal function’ and ‘raised calcium’ as physical_health_recent/longterm even if not phrased as a formal diagnosis. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
    - 'risk' must be one of: ['self_harm', 'risk_to_others', 'general_risk_planning', null]
    - 'clinical_setting' must be one of: ['assessment', 'primary_care', 'secondary_care', 'inpatient', null]
    - 'care_coordination' must be one of: ['current', 'offered', null]
    - 'treatment_response' must be one of: ['good', 'ineffective', 'poorly_tolerated', 'relapse', null]
    - 'test_name' must be the name of a recognised physical health test (e.g., eGFR, urea, creatinine, thyroid). If more than one test is mentioned, record all as a list of strings. If none is mentioned, record null.
    - 'test_result_status' must be one of: ['low', 'raised', 'declining', 'abnormal', 'normal', null]
    - 'patient_preferences' is a string describing what the patient has requested, refused, or expressed a preference for. If no preference is stated, record null.
    - 'advanced_statement_present' must be set to True if the recommendation mentions the possible presence of an official personal plan of care wishes for future crises, False if it is specifically mentioned as absent, or null.

    CONTEXT: {heading_context}

    RECOMMENDATION NUMBER: {original_recommendation_number}
    RECOMMENDATION TEXT: {original_recommendation_text}

    Produce JSON with exactly these keys:
####################################################################### populate list

    REMEMBER:
    - you MUST use null if the information for any field is not explicit in the recommendation or heading
    - if there is more than one value for any field, retain all as a list of strings
    """

In [7]:
# New version: restructuring to "if-then" distinction

def construct_prompt(entity):
    """
    tbc
    """

    heading_1 = entity.get('heading_1')
    sub_heading_1 = entity.get('sub_heading_1')
    sub_heading_2 = entity.get('sub_heading_2')

    original_recommendation_number = entity.get('original_recommendation_number')
    original_recommendation_text = entity.get('original_recommendation_text')

    heading_context = " > ".join(h.strip() for h in [heading_1, sub_heading_1, sub_heading_2] if isinstance(h, str) and h.strip())

    return f"""
You are extracting information from a NICE guideline recommendation into a structured JSON format.

RULES:
- Output valid JSON ONLY (no markdown; no commentary)
- Do not invent clinical information, thresholds, populations or interpretations.
- Prefer phrases that are taken directly from the recommendation text or use only minor rephrasing.
- If something is not explicitly stated, use null.
- If more than one value applies for a field, retain all values as a list of strings.

TASK:
From the recommendation, separate:
1) 'IF' STATE (condition/triggers): everything that constrains when to carry out a clinical action
2) 'THEN' ACTIONS: clinical interventions (assessing, referring, testing, treating, advising, etc.)

'IF' STATE:
- Capture all explicit triggers/constraints indicated by 'if', 'where', 'for people who...', 'in those who...', 'only if...', etc. (e.g., 'if a person has rapid or excessive weight gain, abnormal lipid levels or problems with blood glucose management')
- Each state must be an explicit clause or phrase from the recommendation text (only minor rephasing permitted).
- Store the type of each condition as a separate object in if_state.condition[].
- Do NOT put action verbs or intervention instructions into conditions.

if_state.condition[].category must be one of:
- 'population' is a string of the patient or staff group that the recommendation applies to (e.g., 'people with bipolar…', 'pregnant women…', 'psychological therapists', etc.). If none is mentioned, record null.
- 'age_group' must be one of ['child','young_person','adult','older_adult', null]
- 'sex' must be one of ['female', 'male', 'other', null]
- 'pregnancy' must be True, False, or null if not stated.
- 'diagnoses' must be one or more mental health diagnoses. If more than one diagnosis is mentioned, record all as a list of strings. If none are mentioned, record null.
- 'current_phase' must be one of ['mania','hypomania','depression', 'bipolar_depression', 'mixed','rapid_cycling','euthymic', null]
- 'symptom_severity' must be one of ['mild','moderate','severe', null]
- 'current_psychosis': True if present, False if explicitly negated, or null if not mentioned.
- 'manic_episode_history' must be one of: ['none', 'one', 'multiple', null]
- 'temporal_trigger' must be a string describing time-based constraint on the relevance of an if-condition (e.g., 'during pregnancy', 'in the postnatal period', 'within the first month of treatment', 'for at least 6 months'). If none are mentioned, record null.
- 'prohibition_trigger' from verb phrases including 'do not', 'must not' or 'should not', and clauses beginning 'but...'. If none are mentioned, record null.
- 'prescribed_medication': populate with medication name/names or type/types ONLY if the text explicitly states the person is currently taking the medication
- 'treatment_stage' must be one of: ['start', 'stop', 'continue', 'change', 'adjust', 'monitor', 'other', null]
- treatment_response_summary: one of ['good','ineffective','poorly_tolerated','relapse', 'other', null]
- 'adverse_effect' is a string describing side effects mentioned in the recommendation, or is set to null.
- 'medication_adherence': one of ['good','poor', null]
- 'attitude_towards_psychological_therapy' must be one of: ['positive', 'ambivalent', 'negative', null]
- 'risk_of_selfharm': True if present, False if explicitly negated, or null if not mentioned.
- 'risk_of_suicide': True if present, False if explicitly negated, or null if not mentioned.
- 'risk_to_other_people': True if present, False if explicitly negated, or null if not mentioned.
- 'risk_other_risk_type': True if present, False if explicitly negated, or null if not mentioned.
- physical_health_longterm: physical disease/condition usually >6 months IF explicitly stated
- 'physical_health_recent': recent/transient condition OR explicit abnormal states like 'impaired renal function' or 'raised calcium'
- 'clinical_test_type' must be the name of a recognised physical health test (e.g., eGFR, urea, creatinine, thyroid, etc.).
- 'test_result_status' must be one of: ['low', 'raised', 'declining', 'abnormal', 'normal', 'other', null]
- clinical_setting: one of ['assessment','primary_care','secondary_care','inpatient', null]
- 'care_coordination' must be one of: ['current', 'offered', null]
- 'patient_preferences' is a string describing what the patient has requested, refused, or expressed a preference for. If no preference is stated, record null.
- 'advanced_statement_present' must be set to True if the recommendation mentions the possible presence of an official personal plan of care wishes for future crises, False if it is specifically mentioned as absent, or null.
- 'other'

'IF' LOGIC:
- if_state.logic describes how the categories combine:
  - 'and' if all must hold
  - 'or' if any is sufficient
  - 'undetermined' if mixed/unclear
  - null if there are one or fewer categories
- Do not guess if it is not clear from the wording.

'THEN' ACTIONS:
- Capture each recommended intervention as a separate object in then_actions[].
- The action_text should be a verb phrase (e.g., 'measure the person's BMI', 'continue treatment and care in an early intervention in psychosis service, a specialist bipolar disorder service or a specialist integrated community-based team', 'monitor closely for signs of depression...').
- Action should capture the clinical instruction and not merely the verb (e.g., 'ensure that people have access to calming environments and reduced stimulation', not just 'ensure').
- When an action includes a quantitative target (dose, plasma level, frequency, duration), include that target in the action string.
- 'Do not / must not / should not' statements are actions and must be extracted to actions.prohibitions

then_actions[].action_type must be one of:
- 'information_giving': True if present, False if explicitly negated, or null if not mentioned.
- 'suggested_medications': populate medications that are recommended/considered/started/stopped/changed (if not explicitly current). If the medications are to be taken together (as suggested by 'combined'), record as a single string.
- 'monitoring_test' must be the name of a recognised physical health test (e.g., eGFR, urea, creatinine, thyroid, etc.) if the action is to carry out a test or monitor a parameter; otherwise null.
- 'psychological_therapy_type' must be one of: ['psychodynamic', 'psychoeducation', 'cognitive_behavioural', 'other', null]
- 'psychological_therapy_delivery' must be one of: ['individual', 'group', null]
- 'onward_referral': True if suggested, False if explicitly negated, or null if not mentioned.
- 'care_coordination': True if suggested, False if explicitly negated, or null if not mentioned.
- 'risk_management': True if suggested, False if explicitly negated, or null if not mentioned.
- 'other_treatment' should be a string describing any other type of treatment action not covered by the above categories (e.g., exercise programme, nutrition, electroconvulsive therapy), or null.
- 'urgency': True if 'urgent/urgently/immediate/immediately' present; else False
- 'temporal_constraints' must capture additional requirements attached to the action, such as timing, frequency, duration or review intervals that affects an action (e.g., 'within 4 weeks of symptom resolution', 'for a trial period of 6 months'). If no temporal_constraints are mentioned, record null.

CONTEXT: {heading_context}

RECOMMENDATION NUMBER: {original_recommendation_number}
RECOMMENDATION TEXT: {original_recommendation_text}

Produce JSON with exactly these keys (and no others):


"""


In [ ]:
# Check prompt length

recommendation_for_prompt_check = raw_recommendations[24]

prompt_test = construct_prompt(recommendation_for_prompt_check)

token_count = tokenizer.encode(prompt_test)

print(f"Prompt token length: {len(token_count)}\n")

print(f"Recommendation used for prompt check: {recommendation_for_prompt_check}")

Prompt token length: 1448

Recommendation used for prompt check: {'heading_1': '1.3 Assessing suspected bipolar disorder in adults in secondary care', 'sub_heading_1': None, 'sub_heading_2': None, 'original_recommendation_number': '1.3.2', 'original_recommendation_text': "When assessing suspected bipolar disorder:, and undertake a full psychiatric assessment, documenting a detailed history of mood, episodes of overactivity and disinhibition or other episodic and sustained changes in behaviour, symptoms between episodes, triggers to previous episodes and patterns of relapse, and family history, and assess the development and changing nature of the mood disorder and associated clinical problems throughout the person's life (for example, early childhood trauma, developmental disorder or cognitive dysfunction in later life), and assess social and personal functioning and current psychosocial stressors, and assess for potential mental and physical comorbidities, and assess the person's phys

In [8]:
def run_llm_on_entity(tokenizer, model, entity):

    """
    Call the model on a single prompt using the prompt function
    Return model response
    """

    prompt = construct_prompt(entity)

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             do_sample=False) # deterministic decoding without random sampling
                                            # if removed, reinstate temperature / top_p / top_k

    llm_response = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[1]:],
                                          skip_special_tokens=True)

    return llm_response[0]

Source for nested curly braces extraction: https://til.magmalabs.io/posts/01a278bb48-extracting-json-code-with-nested-curly-braces-in-ruby-the-long-painful-way-around-with-help-from-gpt4

In [9]:
def convert_output_to_true_json(llm_response):
    """
    Takes output from run_llm_on_entity
    Turns it into a true JSON dictionary
    Checks whether it is a JSON file
    """

    llm_response = llm_response.strip()

    start = llm_response.find("{")

    if start == -1: # Where -1 is not found in .find string method
        raise ValueError("In converting_output_to_true_json function, no initial { was found in the output from the LLM.\n")

    brace_count = 0
    json_string = None

    for i in range(start, len(llm_response)):

        if llm_response[i] == "{":
            brace_count += 1
        elif llm_response[i] == "}":
            brace_count -= 1

            if brace_count == 0:

                json_string = llm_response[start:i + 1].strip()
                break

    if json_string is None:
        raise ValueError("In converting_output_to_true_json function, no closing } was found in the output from the LLM.\n")

    try:
        json_object = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        raise

    if not isinstance(json_object, dict):
        raise TypeError(f"The object created by the function convert_output_to_true_json is not a JSON object. Instead it is a {type(json_object)}.")

    return json_object

In [10]:
def validate_json(json_object, required_keys):

    """
    Checks the structure of the JSON to see whether it has the required keys
    ... and is populated with the right types of data
    """

    missing_keys = [k for k in required_keys if k not in json_object]
    if missing_keys:
        raise ValueError(f"Missing required keys: {missing_keys}")

    extra_keys = [k for k in json_object.keys() if k not in required_keys]
    if extra_keys:
        raise ValueError(f"Unexpected extra keys: {extra_keys}")

    for key, value in json_object.items():
        if value is None:
            continue

        # Fields with constrained values
        if key == 'age_group' and value in {'child','young_person','older_adult','adult'}:
            continue
        if key == 'clinical_setting' and value in {'assessment', 'primary_care', 'secondary_care', 'inpatient'}:
            continue
        if key == 'manic_episode_history' and value in {'none', 'one', 'multiple'}:
            continue
        if key == 'conditionality_operator' and value in {'and', 'or'}:
            continue
        if key == 'current_manic_phase' and value in {'mania', 'hypomania', 'depression', 'mixed', 'rapid_cycling', 'euthymic'}:
            continue
        if key == 'symptom_severity' and value in {'mild', 'moderate', 'severe'}:
            continue
        if key == 'current_psychosis' and value in {'present', 'absent'}:
            continue
        if key == 'medication_adherence' and value in {'good', 'poor'}:
            continue
        if key == 'risk' and value in {'self_harm', 'risk_to_others'}:
            continue
        if key == 'care_coordination' and value in {'current', 'offered'}:
            continue

        # Boolean fields
        if key in {"urgency", "psychological_therapy"} and isinstance(value, bool):
            continue

        # String or list of string fields
        if isinstance(value, str):
            continue

        if isinstance(value, list) and all(isinstance(v, str) for v in value):
            continue

        raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")

    return json_object

In [12]:
def validate_json(json_object, required_keys):

    """
    Checks the structure of the JSON to see whether it has the required keys
    ... and is populated with the right types of data
    """

    missing_keys = [k for k in required_keys if k not in json_object]
    if missing_keys:
        raise ValueError(f"Missing required keys: {missing_keys}")

    extra_keys = [k for k in json_object.keys() if k not in required_keys]
    if extra_keys:
        raise ValueError(f"Unexpected extra keys: {extra_keys}")

    for key, value in json_object.items():
        if value is None:
            continue

        # Fields with constrained values (new schema)
        if key == "age_group" and value in {"child", "young_person", "older_adult", "adult"}:
            continue

        if key == "clinical_setting" and value in {"assessment", "primary_care", "secondary_care", "inpatient"}:
            continue

        if key == "current_manic_phase" and value in {"mania", "hypomania", "depression", "mixed", "rapid_cycling", "euthymic"}:
            continue

        if key == "symptom_severity" and value in {"mild", "moderate", "severe"}:
            continue

        if key == "current_psychosis" and value in {"present", "absent"}:
            continue

        if key == "medication_adherence" and value in {"good", "poor"}:
            continue

        if key == "risk" and value in {"self_harm", "risk_to_others", "general_risk_planning"}:
            continue

        if key == "care_coordination" and value in {"current", "offered"}:
            continue

        if key == "treatment_response_summary" and value in {"good", "ineffective", "poorly_tolerated", "relapse"}:
            continue

        if key == "test_result_status" and value in {"low", "raised", "declining", "abnormal", "normal"}:
            continue

        # Structured objects required by new prompt
        if key == "if_state":
            if not isinstance(value, dict):
                raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")
            if set(value.keys()) != {"predicates", "operator"}:
                raise TypeError(f"Key '{key}' must have exactly keys: ['predicates', 'operator'].")
            if not isinstance(value["predicates"], list):
                raise TypeError("if_state.predicates must be a list.")
            for p in value["predicates"]:
                if not isinstance(p, dict):
                    raise TypeError("Each predicate must be an object (dict).")
                if set(p.keys()) != {"text_span", "category", "negated"}:
                    raise TypeError("Each predicate must have exactly keys: ['text_span','category','negated'].")
                # predicate fields can be null, but if present enforce types/allowed categories
                if p["text_span"] is not None and not isinstance(p["text_span"], str):
                    raise TypeError("predicate.text_span must be a string or null.")
                if p["category"] is not None:
                    if not isinstance(p["category"], str):
                        raise TypeError("predicate.category must be a string or null.")
                    if p["category"] not in {
                        "patient_characteristic",
                        "treatment_status",
                        "treatment_response",
                        "adverse_effect",
                        "test_result",
                        "risk",
                        "setting",
                        "preference",
                        "temporal_trigger",
                        "prohibition_trigger",
                        "other",
                    }:
                        raise TypeError(f"predicate.category has unexpected value: {p['category']}")
                if p["negated"] is not None and not isinstance(p["negated"], bool):
                    raise TypeError("predicate.negated must be a bool or null.")
            # operator can be null or one of allowed strings
            op = value["operator"]
            if op is not None and op not in {"and", "or", "undetermined"}:
                raise TypeError("if_state.operator must be 'and', 'or', 'undetermined', or null.")
            continue

        if key == "then_actions":
            if not isinstance(value, list):
                raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")
            for a in value:
                if not isinstance(a, dict):
                    raise TypeError("Each then_actions item must be an object (dict).")
                if set(a.keys()) != {"action_text", "action_type", "timing_frequency", "prohibitions_cautions"}:
                    raise TypeError("Each action must have exactly keys: ['action_text','action_type','timing_frequency','prohibitions_cautions'].")
                if a["action_text"] is not None and not isinstance(a["action_text"], str):
                    raise TypeError("action.action_text must be a string or null.")
                if a["action_type"] is not None:
                    if not isinstance(a["action_type"], str):
                        raise TypeError("action.action_type must be a string or null.")
                    if a["action_type"] not in {
                        "information_giving",
                        "medication",
                        "monitoring_test",
                        "psychological_therapy",
                        "referral",
                        "care_coordination",
                        "environmental",
                        "risk_management",
                        "other",
                    }:
                        raise TypeError(f"action.action_type has unexpected value: {a['action_type']}")
                if a["timing_frequency"] is not None and not isinstance(a["timing_frequency"], str):
                    raise TypeError("action.timing_frequency must be a string or null.")
                pc = a["prohibitions_cautions"]
                if pc is not None:
                    if isinstance(pc, str):
                        pass
                    elif isinstance(pc, list) and all(isinstance(v, str) for v in pc):
                        pass
                    else:
                        raise TypeError("action.prohibitions_cautions must be a string, list of strings, or null.")
            continue

        # Boolean fields (new schema)
        if key in {"urgency"} and isinstance(value, bool):
            continue

        if key == "advanced_statement_present":
            if isinstance(value, bool) or value is None:
                continue
            raise TypeError(f"Key '{key}' must be bool or null.")

        # String or list of string fields (as per prompt)
        if isinstance(value, str):
            continue

        if isinstance(value, list) and all(isinstance(v, str) for v in value):
            continue

        raise TypeError(f"Key '{key}' is of the wrong type, namely: {type(value)}.")

    return json_object


In [ ]:
def orchestrate_create_json(raw_recommendations, tokenizer, model, save_file):

    compiled_recommendations = []

    errors = []

    required_keys = ['heading_context',
                    'original_recommendation_number',
                    'original_recommendation_text',
                    'action',
                    'information_giving',
                     'temporal_constraints'
                    'prohibitions_and_cautions',
                    'population',
                    'age_group',
                    'conditionality',
                     'conditionality_operator',
                    'urgency',
                    'manic_episode_history',
                    'current_manic_phase',
                    'symptom_severity',
                    'current_psychosis',
                    'diagnoses',
                    'current_medication',
                    'suggested_medication',
                    'medication_adherence',
                    'physical_health_longterm',
                    'physical_health_recent',
                    'risk',
                    'psychological_therapy',
                    'other_treatments',
                    'clinical_setting',
                    'care_coordination']

    counter = 0

    for i, entity in enumerate(raw_recommendations): # Add [:] slicing for test runs

        llm_output_text = run_llm_on_entity(tokenizer, model, entity)

        try:
            parsed_json = convert_output_to_true_json(llm_output_text)
            validate_json(parsed_json, required_keys)

        except Exception as e:
            print(f"Error {e} at point {i}")
            errors.append({"index": i, "error": str(e), "raw_llm_output": llm_output_text, "entity": entity})
            continue

        compiled_recommendations.append(parsed_json)

        counter += 1

        if counter % 10 == 0: print(f'Number of recommendations processed: {counter}')

    with open(save_file, "w", encoding="utf-8") as f:
        json.dump(compiled_recommendations, f, ensure_ascii=False, indent=2)

    datetime_now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    ERROR_SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON/LLM_conversion_errors"
    os.makedirs(ERROR_SAVE_PATH, exist_ok=True)
    ERROR_SAVE_FILE = os.path.join(ERROR_SAVE_PATH, f"llm_conversion_errors_{datetime_now}.json")

    with open(ERROR_SAVE_FILE, "w", encoding="utf-8") as f:
        json.dump(errors, f, ensure_ascii=False, indent=2)

    print(f"Here is the list of json parsing errors: {errors}\n\n")

    return compiled_recommendations, errors

In [11]:
def orchestrate_create_json(raw_recommendations, tokenizer, model, save_file):

    compiled_recommendations = []
    errors = []

    # REQUIRED KEYS for the new IF/THEN schema (top-level only)
    required_keys = [
        "heading_context",
        "original_recommendation_number",
        "original_recommendation_text",

        "if_state",
        "then_actions",

        "population",
        "urgency",

        "age_group",
        "diagnoses",

        "current_manic_phase",
        "symptom_severity",
        "current_psychosis",

        "clinical_setting",
        "care_coordination",

        "prescribed_medication",
        "suggested_medications",
        "medication_adherence",

        "side_effects",
        "treatment_response_summary",

        "test_name",
        "test_result_status",

        "patient_preferences",

        "physical_health_longterm",
        "physical_health_recent",

        "risk",
        "advanced_statement_present",
    ]

    counter = 0

    for i, entity in enumerate(raw_recommendations):  # Add [:] slicing for test runs
        llm_output_text = run_llm_on_entity(tokenizer, model, entity)

        try:
            parsed_json = convert_output_to_true_json(llm_output_text)
            validate_json(parsed_json, required_keys)

        except Exception as e:
            print(f"Error {e} at point {i}")
            errors.append(
                {
                    "index": i,
                    "error": str(e),
                    "raw_llm_output": llm_output_text,
                    "entity": entity,
                }
            )
            continue

        compiled_recommendations.append(parsed_json)
        counter += 1

        if counter % 10 == 0:
            print(f"Number of recommendations processed: {counter}")

    with open(save_file, "w", encoding="utf-8") as f:
        json.dump(compiled_recommendations, f, ensure_ascii=False, indent=2)

    datetime_now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    ERROR_SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON/LLM_conversion_errors"
    os.makedirs(ERROR_SAVE_PATH, exist_ok=True)
    ERROR_SAVE_FILE = os.path.join(ERROR_SAVE_PATH, f"llm_conversion_errors_{datetime_now}.json")

    with open(ERROR_SAVE_FILE, "w", encoding="utf-8") as f:
        json.dump(errors, f, ensure_ascii=False, indent=2)

    print(f"Here is the list of json parsing errors: {errors}\n\n")

    return compiled_recommendations, errors


In [16]:
orchestrate_create_json(raw_recommendations, tokenizer, model, SAVE_FILE)

JSON parsing error: Expecting ',' delimiter: line 3 column 40 (char 199)
Error Expecting ',' delimiter: line 3 column 40 (char 199) at point 0
JSON parsing error: Expecting ',' delimiter: line 3 column 40 (char 199)
Error Expecting ',' delimiter: line 3 column 40 (char 199) at point 1
JSON parsing error: Expecting ',' delimiter: line 3 column 40 (char 199)
Error Expecting ',' delimiter: line 3 column 40 (char 199) at point 2


KeyboardInterrupt: 

In [ ]:
# Checked 27/01/2026

## LLM output missing

1.1.4
1.1.5
1.1.9
1.2.2
1.2.5
1.3.2
1.3.5
1.3.7
1.4.1
1.5.1
1.5.2
1.5.3
1.5.7
1.5.10
1.6.3
1.6.4
1.7.1
1.7.4
1.10.7
1.11.8